## Data Prep Pipeline
This notebook is the pipeline that generates the clean data set for the Grandmaster Migration Data App. 

In [1]:
## import packages
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
## read data
data = pd.read_csv('../data/combined/finish.dat', names=['yr_month', 'geo', 'count'])

In [3]:
# strip whitespace from geography
data['geo'] = data['geo'].str.strip()

# splitting out year and month.
data['yr_month_str'] = data['yr_month'].map(str)
data['year']=data['yr_month_str'].str.slice(stop=4)
data['month'] = data['yr_month_str'].str.slice(start=4)

conditions = [
    data['month'].isin(['01','02','03']),
    data['month'].isin(['04','05','06']),
    data['month'].isin(['07','08','09']),
    data['month'].isin(['10','11','12'])]
choices = ['Q1','Q2','Q3','Q4']
data['quarter'] = np.select(conditions,choices, default=0)

data['label_friendly'] = data['year']+'-'+data['quarter']
data.drop(labels=['yr_month_str'], axis=1, inplace=True)
data[['year', 'month']] = data[['year', 'month']].apply(pd.to_numeric)
data.head()

,yr_month,geo,count,year,month,quarter,label_friendly
0,200101,ARM,3,2001,1,Q1,2001-Q1
1,200101,BEL,1,2001,1,Q1,2001-Q1
2,200101,BIH,2,2001,1,Q1,2001-Q1
3,200101,BLR,1,2001,1,Q1,2001-Q1
4,200101,BRA,1,2001,1,Q1,2001-Q1


In [4]:
#read geography shapess
worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

C:\Users\laura\AppData\Local\Temp\ipykernel_24540\246622495.py:2: FutureWarning: The geopandas.dataset module is deprecated and will be removed in GeoPandas 1.0. You can get the original 'naturalearth_lowres' data from https://www.naturalearthdata.com/downloads/110m-cultural-vectors/.
  worldmap = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))


In [5]:
def calculate_center(df):
    """
    Calculate the centre of a geometry

    This method first converts to a planar crs, gets the centroid
    then converts back to the original crs. This gives a more
    accurate
    """
    original_crs = df.crs
    planar_crs = 'EPSG:3857'
    return df['geometry'].to_crs(planar_crs).centroid.to_crs(original_crs)

worldmap["center"] = calculate_center(worldmap)
worldmap["Longitude"] = [val.x for val in worldmap.center]
worldmap["Latitude"] = [val.y for val in worldmap.center]

In [7]:
# country codes that do not correspond to ISO countries, and represents outliers
rm_dict = {'DAI', 'HAO','HUA','IAN','MAX','YUE','JAN','SON','JON','RAY','QUN','FID','JUN'}

# country codes that do not correspond to ISO countries, and represent useful data (non outliers)
fr_dict = {'SAM': 'WSM',
'BUL': 'BGR',
'CHI': 'CHL',
'CRO': 'HRV', 
'DEN': 'DNK',
'ENG': 'GBR',
'GER': 'DEU',
'GRE': 'GRC',
'IRI': 'IRN',
'LAT': 'LVA',
'NED': 'NLD',
'PAR': 'PRY',
'PHI': 'PHL',
'ROM': 'ROU',
'SCG': 'SRB',
'SGP': 'SGP',
'SIN': 'SGP',
'SLO': 'SVN',
'SUI': 'CHE',
'UAE': 'ARE',
'URU': 'URY',
'VIE': 'VNM',
'VAN': 'VUT'}


In [8]:
data.replace({'geo': fr_dict}, inplace=True)

In [9]:
#join worldmap to data
full_data = pd.merge(data, worldmap, left_on='geo', right_on='iso_a3', how='left')

In [10]:
# manual updates for small countries that were missing from the lowres dataset
full_data.loc[full_data.geo == "WSM", ['name', 'iso_a3', 'Longitude', 'Latitude']] = 'Samoa', 'WSM' , 172.1046, -13.7590
full_data.loc[full_data.geo == "SGP", ['name', 'iso_a3', 'Longitude', 'Latitude']] = 'Singapore', 'SGP' , 103.8198, 1.3521

# manual updates for distributed countries where the centroid is not representative on the map
full_data.loc[full_data.geo == "USA", ['Longitude', 'Latitude']] =  -94.5786, 39.0997
full_data.loc[full_data.geo == "CAN", ['Longitude', 'Latitude']] =  -101.3023, 57.5212


In [11]:
full_data = full_data.dropna(how='any').reset_index(drop=True)

In [12]:
df=full_data[['label_friendly', 'count', 'month', 'year', 'name', 'Longitude', 'Latitude']].copy()

In [13]:
df.to_csv('clean_data.csv', index=False)